<a href="https://colab.research.google.com/github/alvumu/TGINE/blob/main/Practica1/3_2_scrapyJSON_LD_Elmundo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sesión 3 - Scrapy - Ejemplo Crawling JSON-LD (ElMundo)

## Apartado 1.1 Crawler de El Mundo extrayendo JSON-LD
En el ejemplo siguiente definimos un crawler en Scrapy para extraer noticias de los metadatos de las páginas web.
El Mundo y otros periódicos publican metadatos en formato JSON-LD (https://json-ld.org/) que permite obtener información estructurada de las webs. En este caso, tendremos que obtener esos objetos JSON-LD y extraer su información en el formato de noticia (https://schema.org/NewsArticle) publicado en Schema.org

** Existen librerías de Python para trabajar directamente con JSON-LD

In [ ]:
!pip3 install -U scrapy

In [ ]:
import scrapy
import sys
import json
import locale
import time
import random
from bs4 import BeautifulSoup

class ElMundoSpider (scrapy.Spider):

    #Es obligatorio poner un nombre
    name = 'elmundo'

    #Ponemos que el dominio que está permitido es el de la página y no vamos a irnos fuera de la misma
    allowed_domains = ['www.elmundo.es']

    start_urls = ['https://www.elmundo.es/']

    # para evitar que el sitio te bloquee por usar scrapy es interesante cambiar el USER_AGENT
    # El user agent por defecto de Scrapy cuando hace una petición es
    # Scrapy/VERSION (+https://scrapy.org)
    custom_settings = {
        'USER_AGENT': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
    }

    def parse (self, response):
      """
      @inherit

      @param self
      @param response
      """

      # Obtenemos la URL de la web que estamos procesando
      url = str(response.request.url).strip()

      # Obtenemos usando CSS la parte donde está definido el JSON-LD en la web
      for item in response.css('script[type=\"application/ld+json\"]'):
        data = json.loads(item.css('::text').get())
        # Procesamos aquí los datos
        # RELLENAR
        if("headline" in data):
          title = data["headline"]
          content = data["description"]
          date = data["datePublished"]

          print ("-------------------------")
          print (url)
          print (title)
          print (content)
          print (date)
          print ("-------------------------")

          data = {
                'url' : url,
                'title': title,
                'content': content,
                'date': date
            }

            #Creamos para cada item un fichero json y para ello obtenemos un número aleatorio.
          filename = str(random.random()).replace(".","") + ".json"

            # Guardamos el documento si tiene contenido y título
          if content and title:
              with open ('elmundo/' + filename, 'w') as f:
                  json.dump (data, f, indent = 4)

      # Por último, miramos distintos enlaces que existan en la web (en este caso todos los de la etiqueta <a>)
      url_in_current_document = response.css ('a')
      for next_page in url_in_current_document:
          # Por cada enlace ejecutamos el mismo parse
          #Intentamos filtrar que solamente coja deportes
          url_str = str(next_page.css("::attr(href)").get())
          if(url_str.find("elmundo.es/deportes") != -1) :
            yield response.follow (next_page, self.parse)


## Apartado 1.2
Para poder lanzar el Spider necesitamos que ejecutar el siguiente código donde se configuará y lanzará el proceso.
Hay que hacer notar que solamente se puede lanzar un proceso por cada sesión en Jupyter notebook es por eso por lo que se recomienda exportar el código en un script de Python .py para poder ejecutarlo desde la línea de comandos.

In [ ]:
import os
import scrapy
from scrapy.crawler import CrawlerProcess

# Creamos un proceso de Crawler podemos poner distintas settings que están definidas en la documentación.
# Entre ellas podemos ocular los logs del proceso de Crawling.
process = CrawlerProcess(settings={
    "LOG_ENABLED": False,
    # Used for pipeline 1
})

# Como se ha definido anteriormente en el RSSCrawler, los ficheros se van a almacenar en la carpeta "rss"
# Comprobamos que existe la carpeta y si no existe la creamos
if (not os.path.exists('elmundo')):
    os.mkdir('elmundo')

# Creamos el proceso con el RSSSpider
process.crawl(ElMundoSpider)
# Ejecutamos el Crawler
process.start()

INFO:scrapy.utils.log:Scrapy 2.11.0 started (bot: scrapybot)
INFO:scrapy.utils.log:Versions: lxml 4.9.3.0, libxml2 2.10.3, cssselect 1.2.0, parsel 1.8.1, w3lib 2.1.2, Twisted 22.10.0, Python 3.10.12 (main, Jun 11 2023, 05:26:28) [GCC 11.4.0], pyOpenSSL 23.2.0 (OpenSSL 3.1.4 24 Oct 2023), cryptography 41.0.5, Platform Linux-5.15.120+-x86_64-with-glibc2.35
INFO:scrapy.addons:Enabled addons:
[]
/usr/local/lib/python3.10/dist-packages/scrapy/utils/request.py:254: ScrapyDeprecationWarning: '2.6' is a deprecated value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting.

It is also the default value. In other words, it is normal to get this warning if you have not defined a value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting. This is so for backward compatibility reasons, but it will change in a future version of Scrapy.

See the documentation of the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting for information on how to handle this deprecation.
  return cls(crawler)
DEBUG:scrap

-------------------------
https://www.elmundo.es/deportes/mas-deporte/2023/10/25/6539588afdddfff51e8b45db.html
Tigist Assefa: &quot;Voy a darlo todo por acercarme lo máximo a los tiempos de los hombres&quot;
Hace exactamente un mes y dos días, Tigist Assefa (Adis Abeba, Etiopía, 1996) destrozó en Berlín el récord del mundo de maratón. Ninguna mujer había bajado jamás de las dos...
2023-10-25T21:57:45Z
-------------------------
-------------------------
https://www.elmundo.es/deportes/futbol/primera-division/2023/10/26/653a4071e9cf4a7b358b45c7.html
Florentino no tiene intención de ir al palco del Barcelona
&quot;A estas horas, no va&quot;. El mensaje que sale del Real Madrid habla de la intención de su presidente, Florentino Pérez, de no acudir el próximo sábado al palco del...
2023-10-26T10:36:19Z
-------------------------
-------------------------
https://www.elmundo.es/deportes/futbol/2023/10/26/653a467a21efa043148b4590.html
El jefe de Marketing de la RFEF pide el archivo y revela nu

DEBUG:scrapy.core.engine:Crawled (200) <GET https://www.elmundo.es/deportes/baloncesto/nba/pag2> (referer: https://www.elmundo.es/deportes/baloncesto/nba.html)
DEBUG:scrapy.core.engine:Crawled (200) <GET https://www.elmundo.es/deportes/baloncesto/mundial/pag2> (referer: https://www.elmundo.es/deportes/baloncesto/mundial.html)
DEBUG:scrapy.core.engine:Crawled (200) <GET https://www.elmundo.es/deportes/baloncesto/eurobasket/2022/09/11/631e134dfdddff94528b4596.html> (referer: https://www.elmundo.es/deportes/baloncesto/eurobasket.html)
DEBUG:scrapy.core.engine:Crawled (200) <GET https://www.elmundo.es/deportes/baloncesto/eurobasket/2022/09/10/631cc1b621efa0d45d8b457f.html> (referer: https://www.elmundo.es/deportes/baloncesto/eurobasket.html)
DEBUG:scrapy.core.engine:Crawled (200) <GET https://www.elmundo.es/deportes/tenis/2023/10/19/65319de0e4d4d8a11f8b45b8.html> (referer: https://www.elmundo.es/deportes.html)


-------------------------
https://www.elmundo.es/deportes/baloncesto/eurobasket/2022/09/11/631e134dfdddff94528b4596.html
Finlandia, rival en cuartos de España, la manada de lobos que lidera Markkanen: &quot;Es algo fuera de lo
La fiesta nacional suomi se desató el domingo en el Mercedes Benz Arena. Por primera vez en 55 años (en 1967 acabaron sextos), Finlandia se coló entre los ocho mejores de un...
2022-09-11T17:10:44Z
-------------------------
-------------------------
https://www.elmundo.es/deportes/baloncesto/eurobasket/2022/09/10/631cc1b621efa0d45d8b457f.html
Lorenzo Brown y una España heroica
Está España donde casi nadie la esperaba, heroica entre los ocho mejores de Europa, rompiendo pronósticos y corazones. En una noche para el recuerdo, para los anales de la...
2022-09-10T21:29:58Z
-------------------------
-------------------------
https://www.elmundo.es/deportes/tenis/2023/10/19/65319de0e4d4d8a11f8b45b8.html
Nadal y su regreso: &quot;Mi primera opción sería en enero, en Aus

DEBUG:scrapy.core.engine:Crawled (200) <GET https://www.elmundo.es/deportes/juegos-olimpicos/2022/02/10/62050c8821efa0137b8b45d1.html> (referer: https://www.elmundo.es/deportes/juegos-olimpicos.html)
DEBUG:scrapy.core.engine:Crawled (200) <GET https://www.elmundo.es/deportes/2023/10/19/65315600e9cf4ab3158b459b.html> (referer: https://www.elmundo.es/deportes.html)
DEBUG:scrapy.core.engine:Crawled (200) <GET https://www.elmundo.es/deportes/baloncesto/eurobasket/2022/09/11/631d12cc21efa0026f8b45c9.html> (referer: https://www.elmundo.es/deportes/baloncesto/eurobasket.html)
DEBUG:scrapy.core.engine:Crawled (200) <GET https://www.elmundo.es/deportes/juegos-olimpicos/2022/02/11/6204ee3721efa0173c8b4600.html> (referer: https://www.elmundo.es/deportes/juegos-olimpicos.html)
DEBUG:scrapy.core.engine:Crawled (200) <GET https://www.elmundo.es/deportes/baloncesto/eurobasket/2022/09/10/631c3536fdddff3a178b45c3.html> (referer: https://www.elmundo.es/deportes/baloncesto/eurobasket.html)
DEBUG:scrapy.c

...

DEBUG:scrapy.core.engine:Crawled (200) <GET https://www.elmundo.es/deportes/mas-deporte/2023/05/17/6464a77521efa09c4d8b4570.html> (referer: https://www.elmundo.es/deportes/mas-deporte/pag3)
DEBUG:scrapy.core.engine:Crawled (200) <GET https://www.elmundo.es/deportes/mas-deporte/2023/05/17/6463bcd1e4d4d830058b45a5.html> (referer: https://www.elmundo.es/deportes/mas-deporte/pag3)


-------------------------
https://www.elmundo.es/deportes/baloncesto/nba/2022/11/14/6371ff1bfc6c83dc388b459d.html
Una noche casi perfecta de Joel Embiid
Con un actuación antológica de 59 puntos, Joel Embiid hizo historia en la NBA este domingo al convertirse en el primer jugador que supera los 50 puntos, 10 rebotes, 5 asistencias...
2022-11-14T08:46:05Z
-------------------------
-------------------------
https://www.elmundo.es/deportes/2023/07/27/64c22a5ffc6c83ef0b8b457b.html
La ajedrecista iraní Sara Khadem que se negó a competir con velo debuta como española en la Copa del Mundo
El domingo 30 empieza en Bakú la Copa del Mundo de Ajedrez, una de las grandes citas del año, que durará casi un mes, hasta el 25 de agosto. El favorito es el número uno,...
2023-07-27T08:47:57Z
-------------------------
-------------------------
https://www.elmundo.es/deportes/baloncesto/eurobasket/2022/09/06/631768bc21efa073088b45a0.html?intcmp=MNOT23801&s_kw=1
El Plan B hace reaccionar a España, que ya est

DEBUG:scrapy.core.engine:Crawled (200) <GET https://www.elmundo.es/deportes/mas-deporte/2023/05/17/6464a038e4d4d8752e8b459a.html> (referer: https://www.elmundo.es/deportes/mas-deporte/pag3)


-------------------------
https://www.elmundo.es/deportes/mas-deporte/2023/05/17/6464a77521efa09c4d8b4570.html
&apos;El hombre del Everest&apos; alcanza las 27 subidas a la montaña más alta del mundo y bate el récord
Kami Rita Sherpa nació en 1970 en Thame, una aldea del Himalaya de donde han salido varios escaladores exitosos, pero ninguno como él. El montañero nepalí alcanzó la cumbre...
2023-05-17T10:07:54Z
-------------------------
-------------------------
https://www.elmundo.es/deportes/mas-deporte/2023/05/17/6463bcd1e4d4d830058b45a5.html
Sara Khadem, la estrella del ajedrez que huyó de Irán: &quot;No pueden detener a todo el mundo&quot;
Sara Khadem (Teherán, 1997) es una joven de su tiempo, feliz y risueña, pero también fuerte y rebelde con causa. Después de una protesta le retiraron el pasaporte y pensó que...
2023-05-17T00:53:11Z
-------------------------
-------------------------
https://www.elmundo.es/deportes/mas-deporte/2023/05/17/6464a038e4d4d8752e8b459a.html
El escalador

DEBUG:scrapy.core.engine:Crawled (200) <GET https://www.elmundo.es/deportes/mas-deporte/pag4> (referer: https://www.elmundo.es/deportes/mas-deporte/pag3)
INFO:scrapy.statscollectors:Dumping Scrapy stats:
{'downloader/request_bytes': 41074,
 'downloader/request_count': 101,
 'downloader/request_method_count/GET': 101,
 'downloader/response_bytes': 6190971,
 'downloader/response_count': 101,
 'downloader/response_status_count/200': 101,
 'dupefilter/filtered': 10249,
 'elapsed_time_seconds': 12.479391,
 'finish_reason': 'shutdown',
 'finish_time': datetime.datetime(2023, 10, 26, 17, 10, 18, 621142, tzinfo=datetime.timezone.utc),
 'httpcompression/response_bytes': 30291467,
 'httpcompression/response_count': 101,
 'log_count/DEBUG': 103,
 'log_count/ERROR': 1,
 'log_count/INFO': 11,
 'memusage/max': 172027904,
 'memusage/startup': 172027904,
 'offsite/filtered': 62,
 'request_depth_max': 6,
 'response_received_count': 101,
 'scheduler/dequeued': 101,
 'scheduler/dequeued/memory': 101,
 's